# 缠论数据图表

In [ ]:
import time

from chanlun import kcharts
from chanlun.cl_utils import query_cl_chart_config
from chanlun.exchange import get_exchange, Market
from chanlun.exchange.exchange_db import ExchangeDB
from chanlun import cl

import itertools
import talib
from chanlun.cl_utils import *
from chanlun.cl_interface import Config


In [ ]:
# 市场
market = 'a'  # a 沪深A股  hk 港股 us 美股 futures 期货 currency 数字货币
# 标的代码
code = 'SH.600036'

# code = 'SZ.000404'  #7.8预警,黄线新低，MACD柱子不创新高
# code = 'SZ.000419' 6.24预警，黄线新低。MACD柱子不创新高
# code = 'sz.000547' 6.24预警，黄线新低，但是MACD柱子在最低位
# code = 'SZ.000532'
# 获取周期，可设置多个
frequencys = ['d']
# frequencys = ['30m','5m']
# 开始时间（需要api接口支持）
start_date = None
# 结束时间（需要api接口支持）
end_date = None
# 缠论计算配置
cl_config = query_cl_chart_config(market, code)
print(cl_config)
ex = get_exchange(Market(market)) # 使用在线的k线数据接口
# ex = ExchangeDB(market) # 读取数据库中的k线数据

In [ ]:
klines[:-800].tail(3)

In [ ]:
klines[:-797].tail(1)
#baocuo 

In [ ]:
klines[:-798].tail(1)

In [ ]:
renders = []
cl_datas = []


for f in frequencys:
    klines = ex.klines(code, frequency=f, start_date=start_date, end_date=end_date)
    klines = klines[:]
    print(f, '获取K线数据量：', len(klines))
    _s = time.time()
    cl_datas.append(cl.CL(code, f, cl_config).process_klines(klines[:-798]))
    print('Run time: ', time.time() - _s)

for cd in cl_datas:
    title = '%s - 【%s】 周期数据图表' % (code, cd.frequency)
    renders.append(kcharts.render_charts(title, cd))

In [ ]:
# 最后一个向下线段
xd = cl_datas[0].xds[-1]
if xd.type == 'up':
    xd = cl_datas[0].xds[-2]
print(xd)
compare_xingcheng_ld(xd.start_line,xd.end_line)

In [ ]:
xd = cl_datas[0].xds[-1]
print(klines[:-797].tail(1))
print(xd.end.done)
print(xd.is_done())
print(xd.end_line.is_done())
print(xd.end_line.end)

In [ ]:
xd = cl_datas[0].xds[-1]
print(xd)
print(xd.low)
line = cl_datas[0].xds[-2]
print(line)
print(line.low)

In [ ]:
line.low > xd.low

In [ ]:
cd = cl_datas[0]
lines = cd.get_xds()
line = lines[-1]
line.low > lines[-3].low

In [ ]:
# 最后一个向下趋势段
xd = cl_datas[0].zsds[-1]
if xd.type == 'up':
    xd = cl_datas[0].zsds[-2]
print(xd)
compare_xingcheng_ld(xd.start_line,xd.end_line)

In [ ]:
for mmd in xd.mmds: 
    print(mmd)

In [ ]:
renders[0]

In [ ]:
renders[1]

In [ ]:
renders[2]

In [ ]:
renders[3]

### 以下可以写自己代码获取所需并分析

In [ ]:
def xingchenglidu(line: LINE) -> list:
    """
    计算线的行程力度（取绝对值）

    行程力度 = dy / dx
        dy = 终点与起点的绝对值差异
        dx = 线段之间的k线数量
    """
    if line.end.val == line.start.val:
        return 0

    dy = abs(line.end.val - line.start.val)
    dx = line.end.k.k_index - line.start.k.k_index
    return [dy, dx, dy/dx]

def compare_xingcheng_ld(one_line: LINE, two_line: LINE):
    """
    比较两个线的力度，后者小于前者，返回 True
    :param one_line:
    :param two_line:
    :return:
    """
    dy1, dx1, ld1 = xingchenglidu(one_line)
    dy2, dx2, ld2 = xingchenglidu(two_line)
    if ( (dy2 > dy1) and  (ld2 < ld1)):
        return True
    else:
        return False